# 🎨 Data Designer 101: Seeding Synthetic Data Generation with an External Dataset

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/3-seeding-with-a-dataset.ipynb) to download this notebook to your computer.

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) are used.


In [2]:
data_designer_client = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[14:52:24] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder)

[14:52:24] [INFO] 👀 Preview generation in progress
[14:52:24] [INFO] ✅ Validation passed
[14:52:24] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[14:52:24] [INFO] 🩺 Running health checks for models...
[14:52:24] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[14:52:25] [INFO]   |-- ✅ Passed!
[14:52:25] [INFO] 🌱 Sampling 10 records from seed dataset
[14:52:25] [INFO]   |-- seed dataset size: 820 records
[14:52:25] [INFO]   |-- sampling strategy: ordered
[14:52:25] [INFO] 🎲 Preparing samplers to generate 10 records across 5 columns
[14:52:25] [INFO] (💾 + 💾) Concatenating 2 datasets
[14:52:25] [INFO] 🧩 Generating column `first_name` from expression
[14:52:25] [INFO] 🧩 Generating column `last_name` from expression
[14:52:25] [INFO] 🧩 Generating column `dob` from expression
[14:52:25] [INFO] 🧩 Generating column `physician` from expression
[14:52:25] [INFO] 📝 Preparing llm-text column generation
[

In [8]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': '50aac647-12af-4b93-94b7-0399056f45c5',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'William',                                                               │
│                    │     'last_name': 'Scott',                                                                  │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Male',                                                                         │
│                    │     'street_number': '9452',                                                               │
│                    │     'street_name': 'Santos Mall',                                                          │
│                    │     'city': 'Angelaberg',                                                                  │
│                    │     'state': 'New Jersey',                                                                 │
│                    │     'postcode': '78736',                                                                   │
│                    │     'age': 59,                                                                             │
│                    │     'birth_date': '1966-09-17',                                                            │
│                    │     'country': 'Reunion',                                                                  │
│                    │     'marital_status': 'separated',                                                         │
│                    │     'education_level': 'some_college',                                                     │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Product designer',                                                      │
│                    │     'phone_number': '885.685.5755

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': '50aac647-12af-4b93-94b7-0399056f45c5...,{'uuid': '9afee564-275a-4cf9-a928-372bffe3e6b8...,PT-137500BC,2024-02-27,2024-03-09,William,Scott,1966-09-17,Dr. Bailey,**Dr. Roger Bailey** \n**Primary Care Physici...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': 'a6a86588-02f6-41b0-a001-d94f80352f82...,{'uuid': 'ff4eedf2-c46d-4a17-b059-b5246ec6d916...,PT-F35F1405,2024-12-18,2025-01-06,Terri,Harris,1961-01-12,Dr. Brown,**Dr. Hannah Brown** \n**Primary Care Physici...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,{'uuid': 'fb2dcd68-dcfe-4c4a-978d-20a1ff69c31d...,{'uuid': '211f10cc-03a9-4f1f-bc24-78793fa0aac1...,PT-6BC179BD,2024-05-22,2024-05-28,Heather,Martinez,1943-12-25,Dr. Schwartz,**Dr. Susan Schwartz** \n**Primary Care Physi...
3,arthritis,I have been having trouble with my muscles and...,{'uuid': '2d006272-4afb-4c03-b204-45cbf39028c9...,{'uuid': 'f4ef0710-7bc9-4139-bf75-17bbd11bedbd...,PT-13E4497E,2024-03-05,2024-03-06,Kim,Washington,1931-01-26,Dr. Mayo,**Dr. Tracy Mayo** \n**Primary Care Physician...
4,dengue,I have been feeling really sick. My body hurts...,{'uuid': '780bf47f-97ab-46b1-b7e6-2eb37199d0b4...,{'uuid': 'f46e7bc9-625d-4a61-97bd-78c676ea6008...,PT-D2760854,2024-01-25,2024-02-17,Roger,Roberts,1960-01-31,Dr. Thompson,"**Dr. Rachel Thompson, MD** \n**Primary Care ..."
5,common cold,I've been feeling really run down and weak. My...,{'uuid': '36aa7656-b714-433a-bdf6-493278358208...,{'uuid': 'ab399fd1-e6b2-49ac-bf9b-e45fcfff8fe4...,PT-7B4C2260,2024-11-23,2024-12-04,Krystal,Taylor,1953-05-28,Dr. Duncan,**Patient Name:** Krystal Taylor \n**Date of ...
6,dengue,I have rashes all over my body. I also have a ...,{'uuid': 'ecb5ae1f-e296-45c9-ae1a-faf58b584249...,{'uuid': '6a09035c-8c63-4af1-8425-a65b615998d9...,PT-3686A9DA,2024-01-27,2024-02-17,Christina,Harris,1962-10-06,Dr. Gutierrez,**Dr. Christopher Gutierrez** \n**Primary Car...
7,dengue,I have been feeling nauseous and have a consta...,{'uuid': 'e4578b5c-d46f-4912-95f3-e36688e5c346...,{'uuid': 'a0e73b7f-47d4-4de6-8e94-27fc7b23467d...,PT-BA551903,2024-08-23,2024-09-14,Sharon,Turner,1975-06-10,Dr. Oliver,**Dr. Shannon Oliver** \n**Primary Care Physi...
8,impetigo,I have a rash around my nose that is red and i...,{'uuid': '7fbe5cbf-607f-44f8-8d41-b2a2e4af42ad...,{'uuid': 'f7767dde-f062-4fc9-931e-52a0f09ff20e...,PT-7AFC385D,2024-01-16,2024-01-30,Teresa,Hunter,1974-09-14,Dr. Perkins,**Dr. Timothy Perkins** \n*Primary Care Physi...
9,drug reaction,I have a rash on my chest and back and it itch...,{'uuid': '02e99ebd-19d4-4ac9-b8a9-ec5dbe337b34...,{'uuid': '5ee9dca2-b6b0-4621-b9f9-ede3fe339a18...,PT-A74544C2,2024-04-03,2024-04-06,Julie,Jackson,1946-04-26,Dr. Miller,**Dr. Michael Miller** \n**Primary Care Physi...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                        9 (90.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=20)

[14:52:45] [INFO] 🎨 Creating Data Designer dataset
[14:52:45] [INFO] ✅ Validation passed
[14:52:45] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[14:52:45] [INFO] 📂 Dataset path '/Users/johnnygreco/projects/nvidia/DataDesigner/docs/notebooks/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/Users/johnnygreco/projects/nvidia/DataDesigner/docs/notebooks/artifacts/dataset_12-02-2025_145245' instead.
[14:52:45] [INFO] 🩺 Running health checks for models...
[14:52:45] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[14:52:46] [INFO]   |-- ✅ Passed!
[14:52:47] [INFO] ⏳ Processing batch 1 of 1
[14:52:47] [INFO] 🌱 Sampling 20 records from seed dataset
[14:52:47] [INFO]   |-- seed dataset size: 820 records
[14:52:47] [INFO]   |-- sampling strategy: ordered
[14:52:47] [INFO] 🎲 Preparing samplers to generate 20 records across 5 columns
[14:52:47] [INFO] (💾 + 💾) Concat

In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 42, 'bachelors_field': 'stem', 'birth_...","{'age': 49, 'bachelors_field': 'business', 'bi...",PT-7DF3BE14,2024-05-08,2024-05-16,Rhonda,Campbell,1983-07-21,Dr. Adams,"**Dr. Anthony Adams, Primary Care Physician** ..."
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 114, 'bachelors_field': 'no_degree', '...","{'age': 112, 'bachelors_field': 'business', 'b...",PT-2A086912,2024-12-15,2024-12-25,Christine,Carr,1911-11-28,Dr. Bailey,**Dr. Michael Bailey** **Primary Care Physic...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 90, 'bachelors_field': 'no_degree', 'b...","{'age': 72, 'bachelors_field': 'education', 'b...",PT-2B656599,2024-09-28,2024-10-21,Alejandro,Williams,1935-02-04,Dr. Moore,**Patient Visit Note** **Date:** 2024-10-21 ...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 64, 'bachelors_field': 'no_degree', 'b...","{'age': 23, 'bachelors_field': 'no_degree', 'b...",PT-A7ABD892,2024-04-01,2024-04-03,Mary,Williams,1961-04-07,Dr. Frank,**Dr. Bob Frank** **Primary Care Physician**...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 101, 'bachelors_field': 'no_degree', '...","{'age': 89, 'bachelors_field': 'business', 'bi...",PT-45A0DAB7,2024-11-08,2024-12-03,Monica,Freeman,1924-02-10,Dr. Richardson,**Dr. Chris Richardson** **Primary Care Phys...


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     12 (60.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    20 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      20 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      20 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      20 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Use Data Designer to generate synthetic data for your specific use case!
